## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [8]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "The Genesis Invitational"
tournament_date = "2/22/2026"  # Ending date of tournament
old_course = "The Riviera Country Club"
tournament_id = "R2026007"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Cognizant Classic in The Palm Beaches"
new_ending_date = "3/1/2026"
new_course = "PGA National (Champion)"
new_season = 2026


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [3]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2026
year = 20260  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Codex/PGA local copy
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2026007 (The Genesis Invitational), year: 20260
✅ 72 new rows added for 'The Genesis Invitational'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2026,2026-02-22,R2026007,The Genesis Invitational,The Riviera Country Club,Jacob Bridgeman,1,1,-5,-7,-7,+1,"$4,000,000.00",700.000
1,2026,2026-02-22,R2026007,The Genesis Invitational,The Riviera Country Club,Kurt Kitayama,T2,2,E,-7,-3,-7,"$1,800,000.00",375.000
2,2026,2026-02-22,R2026007,The Genesis Invitational,The Riviera Country Club,Rory McIlroy,T2,2,-5,-6,-2,-4,"$1,800,000.00",375.000
3,2026,2026-02-22,R2026007,The Genesis Invitational,The Riviera Country Club,Adam Scott,4,4,-1,-8,+1,-8,"$1,000,000.00",325.000
4,2026,2026-02-22,R2026007,The Genesis Invitational,The Riviera Country Club,Aldrich Potgieter,5,5,-3,-3,-6,-3,"$840,000.00",300.000


#### Stats

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!! 
statsYear = 2026

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2026 with 999 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,78.0,0.033,54.0,0.231,102.0,-0.131,99.0,-0.066,97.0,...,310.1,96.0,58.67%,41.0,72.62%,81.0,62.32%,364,.3818,2026
1,Aaron Baddeley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,832,.1151,2026
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,434,.3196,2026
3,Aaron Rai,103.0,-0.315,137.0,-0.415,65.0,0.200,104.0,-0.100,124.0,...,285.5,26.0,64.88%,39.0,72.69%,132.0,55.93%,32,2.5574,2026
4,Abraham Ancer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,452,.3085,2026


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

✅ Inserting 120 new rows into odds table...


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [16]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [17]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [18]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [9]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 9 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
715,2016,PGA National (Champion),010,The Honda Classic,2016-02-28
573,2017,PGA National (Champion),010,The Honda Classic,2017-02-26
430,2018,PGA National (Champion),010,The Honda Classic,2018-02-25
287,2019,PGA National (Champion),010,The Honda Classic,2019-03-03
143,2020,PGA National (Champion),010,The Honda Classic,2020-03-01
0,2021,PGA National (Champion),010,The Honda Classic,2021-03-21
1000,2022,PGA National (Champion),010,The Honda Classic,2022-02-27
1144,2023,PGA National (Champion),R2023010,The Honda Classic,2023-02-23
1288,2024,PGA National (Champion),R2024010,Cognizant Classic in The Palm Beaches,2024-03-03


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [10]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-02-28 — The Honda Classic (551 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,8,277.0,57.1,19.79,2,2016-02-28,The Honda Classic
1,Aaron Wise,1,0,0.0,0.0,0.00,0,2016-02-28,The Honda Classic
2,Abraham Ancer,8,0,0.0,0.0,0.00,0,2016-02-28,The Honda Classic



📆 2017-02-26 — The Honda Classic (532 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,12,7,143.45,58.3,11.95,0,2017-02-26,The Honda Classic
1,Aaron Wise,8,5,0.00,62.5,0.00,4,2017-02-26,The Honda Classic
2,Abraham Ancer,5,3,53.50,60.0,10.70,1,2017-02-26,The Honda Classic



📆 2018-02-25 — The Honda Classic (534 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,7,3,35.09,42.9,5.01,0,2022-02-27,The Honda Classic
1,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-02-27,The Honda Classic
2,Aaron Pike,1,0,0.00,0.0,0.00,0,2022-02-27,The Honda Classic



📆 2023-02-23 — The Honda Classic (579 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2023-02-23,The Honda Classic
1,Aaron Baddeley,11,8,246.23,72.7,22.38,1,2023-02-23,The Honda Classic
2,Aaron Beverly,1,1,0.00,100.0,0.00,1,2023-02-23,The Honda Classic



📆 2024-03-03 — Cognizant Classic in The Palm Beaches (561 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,13,7,219.655,53.8,16.90,3,2024-03-03,Cognizant Classic in The Palm Beaches
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-03-03,Cognizant Classic in The Palm Beaches
2,Aaron Rai,15,10,453.289,66.7,30.22,1,2024-03-03,Cognizant Classic in The Palm Beaches


### Recent Form

In [11]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-02-28 — The Honda Classic (551 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jordan Niebrugge,1,6.0,8.66,2016-02-28,The Honda Classic
1,Anthony Wall,1,12.0,17.31,2016-02-28,The Honda Classic
2,Ashley Chesters,1,12.0,17.31,2016-02-28,The Honda Classic



📆 2017-02-26 — The Honda Classic (532 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jared du Toit,1,9.0,12.98,2017-02-26,The Honda Classic
1,Matthew Southgate,1,12.0,17.31,2017-02-26,The Honda Classic
2,Oscar Fraustro,1,13.0,18.76,2017-02-26,The Honda Classic



📆 2018-02-25 — The Honda Classic (534 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Collin Morikawa,9,13.6,5.91,2022-02-27,The Honda Classic
1,Marcel Siem,1,15.0,21.64,2022-02-27,The Honda Classic
2,Rory McIlroy,6,15.2,7.81,2022-02-27,The Honda Classic



📆 2023-02-23 — The Honda Classic (579 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jon Rahm,12,7.8,3.04,2023-02-23,The Honda Classic
1,Sadom Kaewkanjana,1,11.0,15.87,2023-02-23,The Honda Classic
2,Ryo Hisatsune,1,12.0,17.31,2023-02-23,The Honda Classic



📆 2024-03-03 — Cognizant Classic in The Palm Beaches (561 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,13,8.5,3.22,2024-03-03,Cognizant Classic in The Palm Beaches
1,Grant Forrest,1,11.0,15.87,2024-03-03,Cognizant Classic in The Palm Beaches
2,Rory McIlroy,10,12.9,5.38,2024-03-03,Cognizant Classic in The Palm Beaches


### Course History

In [12]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for The Honda Classic on 2016-02-28


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Adam Hadwin,1,31.0,44.72,2016-02-28,PGA National (Champion),The Honda Classic
1,Alex Cejka,1,90.0,129.84,2016-02-28,PGA National (Champion),The Honda Classic
2,Alex Prugh,1,90.0,129.84,2016-02-28,PGA National (Champion),The Honda Classic



🏌️‍♂️ Course history for The Honda Classic on 2017-02-26


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Adam Hadwin,2,60.5,55.07,2017-02-26,PGA National (Champion),The Honda Classic
1,Adam Scott,1,1.0,1.44,2017-02-26,PGA National (Champion),The Honda Classic
2,Alan Morin,1,90.0,129.84,2017-02-26,PGA National (Champion),The Honda Classic



🏌️‍♂️ Course history for The Honda Classic on 2018-02-25


...

,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Wise,3,27.0,19.48,2022-02-27,PGA National (Champion),The Honda Classic
1,Abraham Ancer,1,90.0,129.84,2022-02-27,PGA National (Champion),The Honda Classic
2,Adam Hadwin,3,43.0,31.02,2022-02-27,PGA National (Champion),The Honda Classic



🏌️‍♂️ Course history for The Honda Classic on 2023-02-23


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Rai,1,66.0,95.22,2023-02-23,PGA National (Champion),The Honda Classic
1,Aaron Wise,4,42.8,26.59,2023-02-23,PGA National (Champion),The Honda Classic
2,Abraham Ancer,1,90.0,129.84,2023-02-23,PGA National (Champion),The Honda Classic



🏌️‍♂️ Course history for Cognizant Classic in The Palm Beaches on 2024-03-03


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2024-03-03,PGA National (Champion),Cognizant Classic in The Palm Beaches
1,Aaron Rai,2,78.0,71.00,2024-03-03,PGA National (Champion),Cognizant Classic in The Palm Beaches
2,Aaron Wise,5,40.0,22.32,2024-03-03,PGA National (Champion),Cognizant Classic in The Palm Beaches


In [8]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

## Training Dataset

**Lookahead Avoidance (Stats Source)**

PGA Tour season stats are full-season aggregates, so using same-season stats for historical
events leaks future information (lookahead bias). To avoid this, the pipeline now defaults
to using **prior-season stats** for both training and current-week prediction, while
keeping *current-season form* via features computed from the tournament results table
(recent form, cuts made, FedEx points, course history, etc.).

If you want to change this behavior, adjust `stats_season_offset` or
`fallback_current_stats` in `build_training_rows` / `build_test_rows`.


In [13]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows

# Strict anti-leakage setting:
# - use prior-season stats
# - do not fall back to same-season stats when prior season is missing
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
    stats_season_offset=1,
    fallback_current_stats=False,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-02-28,The Honda Classic,PGA National (Champion),Adam Scott,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,85.7,494.17,70.60,3.0,38.6,18.56,NaN,NaN,1
1,2016,2016-02-28,The Honda Classic,PGA National (Champion),Sergio Garcia,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,85.7,359.93,51.42,0.0,38.4,18.47,31.0,44.72,1
2,2016,2016-02-28,The Honda Classic,PGA National (Champion),Blayne Barber,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.5,74.42,5.72,0.0,77.2,29.25,58.0,83.68,1
3,2016,2016-02-28,The Honda Classic,PGA National (Champion),Justin Thomas,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,69.2,624.33,48.03,1.0,49.5,18.76,90.0,129.84,1
4,2016,2016-02-28,The Honda Classic,PGA National (Champion),Graeme McDowell,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,55.6,800.13,88.90,0.0,51.0,22.15,90.0,129.84,1
5,2016,2016-02-28,The Honda Classic,PGA National (Champion),Rickie Fowler,T6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,70.0,1061.50,106.15,1.0,37.6,15.68,41.0,59.15,1
6,2016,2016-02-28,The Honda Classic,PGA National (Champion),Vijay Singh,T6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.7,280.50,23.38,1.0,52.9,20.62,90.0,129.84,1
7,2016,2016-02-28,The Honda Classic,PGA National (Champion),Billy Horschel,T8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,83.3,546.50,45.54,3.0,37.3,14.54,90.0,129.84,1
8,2016,2016-02-28,The Honda Classic,PGA National (Champion),William McGirt,T8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.7,667.63,47.69,1.0,34.3,12.67,22.0,31.74,1
9,2016,2016-02-28,The Honda Classic,PGA National (Champion),Scott Brown,T10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.1,397.92,28.42,1.0,58.2,21.49,90.0,129.84,1


In [14]:
# Play with different years to see if I notice anything wrong (check Odds for tournament name mismatches)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
1146,2024,2024-03-03,Cognizant Classic in The Palm Beaches,PGA National (Champion),Austin Eckroat,1,1,63.0,0.306,35.0,0.341,104.0,-0.002,126.0,-0.034,74.0,0.112,118.0,21.22%,48.0,3.02,64.0,4.01,127.0,4.65,16.0,120.0,85.0,301.6,35.0,63.94%,105.0,66.61%,56.0,61.59%,98.0,1.1691,100.0,64.7,321.246,18.90,1.0,53.5,18.51,90.0,129.84,1
1147,2024,2024-03-03,Cognizant Classic in The Palm Beaches,PGA National (Champion),Min Woo Lee,T2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,43.0,2.1487,35.0,100.0,53.564,6.70,8.0,28.9,13.15,26.0,37.51,1
1148,2024,2024-03-03,Cognizant Classic in The Palm Beaches,PGA National (Champion),Erik van Rooyen,T2,2,138.0,-0.247,82.0,0.063,96.0,0.056,178.0,-0.367,133.0,-0.168,24.0,23.43%,109.0,3.06,118.0,4.03,38.0,4.57,69.0,178.0,61.0,304.9,117.0,58.06%,53.0,68.38%,186.0,53.15%,85.0,1.2738,60.0,58.8,888.185,52.25,1.0,52.6,18.20,80.0,57.71,1
1149,2024,2024-03-03,Cognizant Classic in The Palm Beaches,PGA National (Champion),David Skinns,T4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,245.0,0.5624,500.0,0.0,0.000,0.00,0.0,90.0,55.92,90.0,129.84,1
1150,2024,2024-03-03,Cognizant Classic in The Palm Beaches,PGA National (Champion),Shane Lowry,T4,4,20.0,0.937,29.0,0.408,28.0,0.461,80.0,0.067,115.0,-0.114,159.0,20.07%,76.0,3.04,155.0,4.05,67.0,4.60,12.0,105.0,71.0,303.8,34.0,64.00%,112.0,66.31%,157.0,56.65%,35.0,2.4640,40.0,80.0,246.275,24.63,2.0,42.6,17.77,22.6,12.61,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [15]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-02-28,The Honda Classic,PGA National (Champion),Adam Scott,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.09,20.0,85.7,494.17,70.60,3.0,38.6,18.56,NaN,NaN,1
1,2016,2016-02-28,The Honda Classic,PGA National (Champion),Sergio Garcia,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.09,40.0,85.7,359.93,51.42,0.0,38.4,18.47,31.0,44.72,1
2,2016,2016-02-28,The Honda Classic,PGA National (Champion),Blayne Barber,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.09,1000.0,38.5,74.42,5.72,0.0,77.2,29.25,58.0,83.68,1
3,2016,2016-02-28,The Honda Classic,PGA National (Champion),Justin Thomas,T3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.09,60.0,69.2,624.33,48.03,1.0,49.5,18.76,90.0,129.84,1
4,2016,2016-02-28,The Honda Classic,PGA National (Champion),Graeme McDowell,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,0.09,60.0,55.6,800.13,88.90,0.0,51.0,22.15,90.0,129.84,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [16]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-02-28,The Honda Classic,PGA National (Champion),Adam Scott,1,1,97.617647,0.09079,96.839572,0.030584,97.36631,0.047737,98.762032,0.013905,95.640374,0.031774,97.53877,20.900722,89.632844,3.056956,88.266043,4.03139,93.883845,4.644793,94.799465,190.569519,97.814171,295.783556,92.755348,61.006872,93.659091,66.367139,98.558824,58.809813,1000.0,0.09,20.0,85.7,494.17,70.60,3.0,38.6,18.56,57.919298,56.465872,1
1,2016,2016-02-28,The Honda Classic,PGA National (Champion),Sergio Garcia,2,2,97.617647,0.09079,96.839572,0.030584,97.36631,0.047737,98.762032,0.013905,95.640374,0.031774,97.53877,20.900722,89.632844,3.056956,88.266043,4.03139,93.883845,4.644793,94.799465,190.569519,97.814171,295.783556,92.755348,61.006872,93.659091,66.367139,98.558824,58.809813,1000.0,0.09,40.0,85.7,359.93,51.42,0.0,38.4,18.47,31.000000,44.720000,1
2,2016,2016-02-28,The Honda Classic,PGA National (Champion),Blayne Barber,T3,3,97.617647,0.09079,96.839572,0.030584,97.36631,0.047737,98.762032,0.013905,95.640374,0.031774,97.53877,20.900722,89.632844,3.056956,88.266043,4.03139,93.883845,4.644793,94.799465,190.569519,97.814171,295.783556,92.755348,61.006872,93.659091,66.367139,98.558824,58.809813,1000.0,0.09,1000.0,38.5,74.42,5.72,0.0,77.2,29.25,58.000000,83.680000,1
3,2016,2016-02-28,The Honda Classic,PGA National (Champion),Justin Thomas,T3,3,97.617647,0.09079,96.839572,0.030584,97.36631,0.047737,98.762032,0.013905,95.640374,0.031774,97.53877,20.900722,89.632844,3.056956,88.266043,4.03139,93.883845,4.644793,94.799465,190.569519,97.814171,295.783556,92.755348,61.006872,93.659091,66.367139,98.558824,58.809813,1000.0,0.09,60.0,69.2,624.33,48.03,1.0,49.5,18.76,90.000000,129.840000,1
4,2016,2016-02-28,The Honda Classic,PGA National (Champion),Graeme McDowell,5,5,97.617647,0.09079,96.839572,0.030584,97.36631,0.047737,98.762032,0.013905,95.640374,0.031774,97.53877,20.900722,89.632844,3.056956,88.266043,4.03139,93.883845,4.644793,94.799465,190.569519,97.814171,295.783556,92.755348,61.006872,93.659091,66.367139,98.558824,58.809813,1000.0,0.09,60.0,55.6,800.13,88.90,0.0,51.0,22.15,90.000000,129.840000,1


In [17]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [123]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [124]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2026 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Ben Griffin,10500
1,Jacob Bridgeman,10200
2,Shane Lowry,9900
3,Ryan Gerard,9700
4,Adam Scott,9600
5,Rasmus Hojgaard,9500
6,Nicolai Hojgaard,9400
7,Keith Mitchell,9300
8,Michael Thorbjornsen,9200
9,Rasmus Neergaard-Petersen,9100


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Cognizant Classic in The Palm Beaches,Ryan Gerard,16/1,16.0
2,2025,Cognizant Classic in The Palm Beaches,Shane Lowry,16/1,16.0
3,2025,Cognizant Classic in The Palm Beaches,Nicolai Hojgaard,20/1,20.0
4,2025,Cognizant Classic in The Palm Beaches,Rasmus Hojgaard,20/1,20.0
5,2025,Cognizant Classic in The Palm Beaches,Michael Thorbjornsen,25/1,25.0
6,2025,Cognizant Classic in The Palm Beaches,Keith Mitchell,30/1,30.0
7,2025,Cognizant Classic in The Palm Beaches,Brooks Koepka,30/1,30.0
8,2025,Cognizant Classic in The Palm Beaches,Daniel Berger,30/1,30.0
9,2025,Cognizant Classic in The Palm Beaches,Rasmus Neergaard-Petersen,30/1,30.0
10,2025,Cognizant Classic in The Palm Beaches,Christiaan Bezuidenhout,40/1,40.0


## Cut Percentage and FedEx Points

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,5,3,42.792,60.0,8.56,3,2026-03-01,Cognizant Classic in The Palm Beaches
1,Aaron Baddeley,8,3,25.838,37.5,3.23,1,2026-03-01,Cognizant Classic in The Palm Beaches
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2026-03-01,Cognizant Classic in The Palm Beaches
3,Aaron Rai,12,10,408.375,83.3,34.03,2,2026-03-01,Cognizant Classic in The Palm Beaches
4,Aaron Wise,7,1,6.333,14.3,0.90,0,2026-03-01,Cognizant Classic in The Palm Beaches
5,Adam Hadwin,16,9,140.425,56.2,8.78,0,2026-03-01,Cognizant Classic in The Palm Beaches
6,Adam Long,1,0,0.000,0.0,0.00,0,2026-03-01,Cognizant Classic in The Palm Beaches
7,Adam Schenk,21,11,771.514,52.4,36.74,1,2026-03-01,Cognizant Classic in The Palm Beaches
8,Adam Scott,12,11,637.407,91.7,53.12,7,2026-03-01,Cognizant Classic in The Palm Beaches
9,Adam Svensson,18,6,140.864,33.3,7.83,0,2026-03-01,Cognizant Classic in The Palm Beaches


## Recent Form

In [21]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,13,4.0,1.52,2026-03-01,Cognizant Classic in The Palm Beaches
1,Jason Scrivener,1,8.0,11.54,2026-03-01,Cognizant Classic in The Palm Beaches
2,Tyrrell Hatton,2,10.0,9.10,2026-03-01,Cognizant Classic in The Palm Beaches
3,Doc Redman,3,15.7,11.33,2026-03-01,Cognizant Classic in The Palm Beaches
4,Robin Williams,1,17.0,24.53,2026-03-01,Cognizant Classic in The Palm Beaches
5,Andy Sullivan,1,17.0,24.53,2026-03-01,Cognizant Classic in The Palm Beaches
6,Russell Henley,12,18.7,7.29,2026-03-01,Cognizant Classic in The Palm Beaches
7,Rory McIlroy,8,19.0,8.65,2026-03-01,Cognizant Classic in The Palm Beaches
8,Hideki Matsuyama,17,19.5,6.75,2026-03-01,Cognizant Classic in The Palm Beaches
9,Tommy Fleetwood,9,19.6,8.51,2026-03-01,Cognizant Classic in The Palm Beaches


## Course History

In [22]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,2,90.0,81.92,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
1,Aaron Rai,2,78.0,71.00,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
2,Aaron Wise,4,41.8,25.97,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
3,Adam Hadwin,1,8.0,11.54,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
4,Adam Long,4,74.3,46.17,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
5,Adam Schenk,7,62.0,29.82,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
6,Adam Scott,2,51.5,46.88,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
7,Adam Svensson,5,59.4,33.15,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
8,Adrian Meronk,1,14.0,20.20,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches
9,Adrien Dumont de Chassart,1,90.0,129.84,2026-03-01,PGA National (Champion),Cognizant Classic in The Palm Beaches


## Merged Dataframe

In [23]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"],
    stats_season_offset=1,
    fallback_current_stats=True
)

this_week.head()


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,A.J. Ewart,78.0,0.033,54.0,0.231,102.0,-0.131,99.0,-0.066,97.0,-0.087,100.0,23.41%,70.0,3.02,104.0,4.01,54.0,4.46,52.0,145,49.0,310.1,96.0,58.67%,41.0,72.62%,81.0,62.32%,364.0,.3818,2026.0,250.0,60.0,42.792,8.56,3.0,60.2,33.60,NaN,NaN,6300
1,Aaron Rai,103.0,-0.315,137.0,-0.415,65.0,0.200,104.0,-0.100,124.0,-0.432,145.0,20.83%,57.0,3.00,146.0,4.07,70.0,4.50,114.0,195,169.0,285.5,26.0,64.88%,39.0,72.69%,132.0,55.93%,32.0,2.5574,2026.0,35.0,83.3,408.375,34.03,2.0,42.3,16.49,78.0,71.00,8800
2,Aaron Wise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6100
3,Adam Hadwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.0,.6715,2026.0,150.0,56.2,140.425,8.78,0.0,67.5,23.82,8.0,11.54,6700
4,Adam Schenk,144.0,-1.076,152.0,-0.689,132.0,-0.553,58.0,0.166,158.0,-0.970,144.0,21.11%,155.0,3.18,161.0,4.12,146.0,4.65,164.0,265,122.0,300.1,143.0,53.33%,157.0,63.33%,159.0,51.52%,136.0,1.0035,2026.0,300.0,52.4,771.514,36.74,1.0,62.0,20.06,62.0,29.82,6200


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [24]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [25]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [102]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [26]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,A.J. Ewart,78.000000,0.033000,54.000000,0.231000,102.000000,-0.131000,99.000000,-0.066000,97.00000,-0.087000,100.000000,23.410000,70.000000,3.02000,104.000000,4.01000,54.000000,4.460000,52.000000,145.000000,49.000000,310.100000,96.000000,58.670000,41.000000,72.62000,81.000000,62.32000,364.0,0.3818,2026.0,250.0,60.000000,42.792,8.560000,3.000000,60.2,33.600000,55.839759,48.564819,6300
1,Aaron Rai,103.000000,-0.315000,137.000000,-0.415000,65.000000,0.200000,104.000000,-0.100000,124.00000,-0.432000,145.000000,20.830000,57.000000,3.00000,146.000000,4.07000,70.000000,4.500000,114.000000,195.000000,169.000000,285.500000,26.000000,64.880000,39.000000,72.69000,132.000000,55.93000,32.0,2.5574,2026.0,35.0,83.300000,408.375,34.030000,2.000000,42.3,16.490000,78.000000,71.000000,8800
2,Aaron Wise,98.930693,-0.417356,94.405941,-0.152277,94.782178,-0.189119,93.425743,-0.075901,91.19802,-0.098762,101.485149,22.913762,91.851485,3.05901,96.346535,4.01396,93.514851,4.546733,89.564356,176.415842,85.108911,304.940594,91.306931,58.587129,90.762376,68.65604,97.871287,59.26099,1000.0,4.2421,2026.0,1000.0,54.543925,0.000,18.982336,1.485981,90.0,25.631028,55.839759,48.564819,6100
3,Adam Hadwin,98.930693,-0.417356,94.405941,-0.152277,94.782178,-0.189119,93.425743,-0.075901,91.19802,-0.098762,101.485149,22.913762,91.851485,3.05901,96.346535,4.01396,93.514851,4.546733,89.564356,176.415842,85.108911,304.940594,91.306931,58.587129,90.762376,68.65604,97.871287,59.26099,209.0,0.6715,2026.0,150.0,56.200000,140.425,8.780000,0.000000,67.5,23.820000,8.000000,11.540000,6700
4,Adam Schenk,144.000000,-1.076000,152.000000,-0.689000,132.000000,-0.553000,58.000000,0.166000,158.00000,-0.970000,144.000000,21.110000,155.000000,3.18000,161.000000,4.12000,146.000000,4.650000,164.000000,265.000000,122.000000,300.100000,143.000000,53.330000,157.000000,63.33000,159.000000,51.52000,136.0,1.0035,2026.0,300.0,52.400000,771.514,36.740000,1.000000,62.0,20.060000,62.000000,29.820000,6200
5,Adam Scott,12.000000,1.278000,3.000000,0.843000,26.000000,0.591000,118.000000,-0.156000,76.00000,0.092000,55.000000,25.350000,7.000000,2.89000,56.000000,3.96000,14.000000,4.370000,4.000000,62.000000,33.000000,313.100000,29.000000,64.730000,35.000000,72.92000,25.000000,67.95000,52.0,1.9732,2026.0,1000.0,91.700000,637.407,53.120000,7.000000,34.4,13.410000,51.500000,46.880000,9600
6,Adam Svensson,66.000000,0.225000,135.000000,-0.387000,39.000000,0.454000,59.000000,0.158000,155.00000,-0.899000,71.000000,24.690000,57.000000,3.00000,139.000000,4.06000,12.000000,4.360000,141.000000,229.000000,150.000000,295.300000,79.000000,60.320000,148.000000,64.81000,113.000000,57.89000,341.0,0.4058,2026.0,200.0,33.300000,140.864,7.830000,0.000000,71.4,24.250000,59.400000,33.150000,6500
7,Adrien Dumont De Chassart,98.930693,-0.417356,94.405941,-0.152277,94.782178,-0.189119,93.425743,-0.075901,91.19802,-0.098762,101.485149,22.913762,91.851485,3.05901,96.346535,4.01396,93.514851,4.546733,89.564356,176.415842,85.108911,304.940594,91.306931,58.587129,90.762376,68.65604,97.871287,59.26099,1000.0,4.2421,2026.0,1000.0,54.543925,0.000,18.982336,1.485981,90.0,25.631028,55.839759,48.564819,6800
8,Adrien Saddier,112.000000,-0.472000,103.000000,-0.164000,116.000000,-0.255000,95.000000,-0.053000,74.00000,0.112000,113.000000,22.690000,142.000000,3.13000,84.000000,3.99000,136.000000,4.630000,60.000000,155.000000,110.000000,301.600000,45.000000,63.690000,103.000000,68.06000,126.000000,56.52000,85.0,1.4585,2026.0,125.0,50.000000,7.650,1.280000,1.000000,75.3,38.700000,55.839759,48.5648

## Correlations
**Last 4 Years**

In [21]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [27]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [28]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest (Production Evaluation)

Season-aware out-of-fold (OOF) validation by `SEASON` to avoid leakage and
estimate realistic weekly performance.


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss
from sklearn.base import clone
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import numpy as np

# ------------------------------------------------------------------
# Season-aware out-of-fold evaluation (no leakage)
# ------------------------------------------------------------------
X_rf = training_df[feature_cols].copy()
y_rf = training_df["TOP_20"].astype(int).copy()
groups = training_df["SEASON"].astype(int).copy()

n_unique_seasons = groups.nunique()
if n_unique_seasons < 2:
    raise ValueError("Need at least 2 seasons for season-aware cross-validation.")

n_splits = min(5, n_unique_seasons)
cv = GroupKFold(n_splits=n_splits)

rf_cv_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("smote", SMOTE(sampling_strategy=0.5, k_neighbors=2, random_state=42)),
    ("under", RandomUnderSampler(sampling_strategy=0.5, random_state=42)),
    ("rf", RandomForestClassifier(
        n_estimators=500,
        max_depth=8,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1,
    )),
])

# Build out-of-fold predictions for realistic metric estimates.
oof_prob = np.full(len(X_rf), np.nan, dtype=float)

for train_idx, valid_idx in cv.split(X_rf, y_rf, groups):
    model_fold = clone(rf_cv_pipeline)
    X_train, X_valid = X_rf.iloc[train_idx], X_rf.iloc[valid_idx]
    y_train = y_rf.iloc[train_idx]

    model_fold.fit(X_train, y_train)
    oof_prob[valid_idx] = model_fold.predict_proba(X_valid)[:, 1]

mask = np.isfinite(oof_prob)
auc_oof = roc_auc_score(y_rf[mask], oof_prob[mask])
brier_oof = brier_score_loss(y_rf[mask], oof_prob[mask])
logloss_oof = log_loss(y_rf[mask], np.clip(oof_prob[mask], 1e-6, 1 - 1e-6))

print("Season-aware OOF metrics (production baseline)")
print(f"Seasons used: {sorted(groups.unique().tolist())}")
print(f"ROC AUC : {auc_oof:.3f}")
print(f"Brier   : {brier_oof:.4f}  (lower is better)")
print(f"LogLoss : {logloss_oof:.4f}  (lower is better)")


Season-aware OOF metrics (production baseline)
Seasons used: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
ROC AUC : 0.637
Brier   : 0.1458  (lower is better)
LogLoss : 0.4644  (lower is better)


### Final Training + Feature Importance

Train final model on all available historical data, then export this week's
ranked probabilities for optimizer input.


In [30]:
import pandas as pd
import plotly.express as px

# Fit final production model on all training rows
final_model = clone(rf_cv_pipeline)
final_model.fit(X_rf, y_rf)

# Feature importances (from fitted RF inside pipeline)
rf_step = final_model.named_steps["rf"]
importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": rf_step.feature_importances_
}).sort_values(by="Importance", ascending=False)

fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this trained model to score this week's player pool.


In [31]:
# === Prepare test data (this week's players) ===
X_test = this_week[feature_cols].copy()

# Raw model output for maximum ranking separation
this_week["PROBABILITY"] = final_model.predict_proba(X_test)[:, 1]

# Sort highest probability first for optimizer export
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE",
    "OWGR_RANK", "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]
export_df = this_week_sorted[columns_to_show].copy()

print(f"Sum(PROBABILITY): {export_df['PROBABILITY'].sum():.2f}")
print(f"Range(PROBABILITY): {export_df['PROBABILITY'].min():.3f} to {export_df['PROBABILITY'].max():.3f}")

export_df.head(20)


Sum(PROBABILITY): 42.21
Range(PROBABILITY): 0.161 to 0.639


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Doug Ghim,7400,0.638797,53.3,59.400000,70.0,68.4,168.0,335.528,74.000000,27.000000,145.000000,32.000000,62.00000,90.000000,98.000000,126.000000,12.000000,54.000000,65.000000,83.000000,86.000000,117.000000
1,Davis Thompson,8600,0.614111,52.9,68.500000,35.0,64.7,114.0,309.175,20.000000,44.000000,21.000000,87.000000,162.00000,117.000000,42.000000,91.000000,131.000000,40.000000,43.000000,83.000000,33.000000,60.000000
2,Max Homa,7900,0.591982,50.6,20.000000,50.0,70.6,150.0,363.546,93.000000,80.000000,74.000000,132.000000,80.00000,117.000000,87.000000,104.000000,88.000000,35.000000,51.000000,72.000000,101.000000,48.000000
3,Alex Smalley,8300,0.591544,52.3,54.300000,40.0,60.0,119.0,526.348,57.000000,50.000000,76.000000,91.000000,56.00000,50.000000,87.000000,40.000000,70.000000,8.000000,20.000000,56.000000,45.000000,96.000000
4,Ricky Castillo,7500,0.589958,59.2,56.000000,70.0,50.0,161.0,368.737,96.000000,48.000000,134.000000,69.000000,47.00000,7.000000,107.000000,56.000000,16.000000,13.000000,28.000000,57.000000,103.000000,112.000000
5,Michael Brennan,7200,0.584660,51.6,55.839759,50.0,87.5,42.0,570.523,55.000000,17.000000,62.000000,149.000000,156.00000,60.000000,134.000000,122.000000,7.000000,27.000000,2.000000,108.000000,102.000000,135.000000
6,Keith Mitchell,9300,0.578908,49.4,34.000000,30.0,77.8,124.0,323.140,31.000000,16.000000,79.000000,89.000000,132.00000,81.000000,24.000000,84.000000,68.000000,30.000000,23.000000,90.000000,29.000000,71.000000
7,Ryan Gerard,9700,0.571430,40.0,14.500000,16.0,85.7,26.0,1133.826,19.000000,46.000000,8.000000,121.000000,53.00000,24.000000,18.000000,22.000000,25.000000,54.000000,95.000000,53.000000,27.000000,30.000000
8,Haotong Li,8100,0.569382,49.7,55.839759,45.0,57.1,72.0,139.214,8.000000,7.000000,24.000000,37.000000,129.00000,78.000000,134.000000,22.000000,54.000000,17.000000,62.000000,28.000000,17.000000,19.000000
9,Will Zalatoris,8400,0.569256,40.0,55.839759,40.0,66.7,252.0,100.833,10.000000,29.000000,25.000000,35.000000,89.00000,12.000000,28.000000,8.000000,70.000000,53.000000,75.000000,72.000000,57.000000,13.000000


### Save CSV for Optimizer


In [32]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

export_df = export_df.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
